In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import warnings 
warnings.filterwarnings('ignore')

In [3]:
pip install folium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import folium

In [5]:
gt = pd.read_csv('datasets\\Global_terrorist.csv', encoding='ISO-8859-1')

In [6]:
gt

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,201712310022,2017,12,31,NaN,0,NaN,182,Somalia,11,...,NaN,"""Somalia: Al-Shabaab Militants Attack Army Che...","""Highlights: Somalia Daily Media Highlights 2 ...","""Highlights: Somalia Daily Media Highlights 1 ...",START Primary Collection,0,0,0,0,NaN
181687,201712310029,2017,12,31,NaN,0,NaN,200,Syria,10,...,NaN,"""Putin's 'victory' in Syria has turned into a ...","""Two Russian soldiers killed at Hmeymim base i...","""Two Russian servicemen killed in Syria mortar...",START Primary Collection,-9,-9,1,1,NaN
181688,201712310030,2017,12,31,NaN,0,NaN,160,Philippines,5,...,NaN,"""Maguindanao clashes trap tribe members,"" Phil...",NaN,NaN,START Primary Collection,0,0,0,0,NaN
181689,201712310031,2017,12,31,NaN,0,NaN,92,India,6,...,NaN,"""Trader escapes grenade attack in Imphal,"" Bus...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN


In [7]:
gt['country_txt']=='Philippines'

0         False
1         False
2          True
3         False
4         False
          ...  
181686    False
181687    False
181688     True
181689    False
181690     True
Name: country_txt, Length: 181691, dtype: bool

In [8]:
filter1 = gt[gt['country_txt']=='Philippines']

In [9]:
filter1['iyear'] == 2017

2         False
26        False
39        False
96        False
150       False
          ...  
181677     True
181678     True
181681     True
181688     True
181690     True
Name: iyear, Length: 6908, dtype: bool

In [10]:
filter2 = filter1[filter1['iyear'] == 2017]

In [28]:
filter2['imonth'].value_counts()

imonth
3     81
5     71
7     68
12    63
2     62
6     59
9     56
8     51
10    48
11    48
1     44
4     41
Name: count, dtype: int64

In [29]:
filter2['imonth'] <= 6

170847     True
170857     True
170888     True
170933     True
170957     True
          ...  
181677    False
181678    False
181681    False
181688    False
181690    False
Name: imonth, Length: 692, dtype: bool

In [30]:
filter3 = filter2[filter2['imonth'] <= 6]

In [31]:
filter3['attacktype1_txt'].value_counts()

attacktype1_txt
Assassination    358
Name: count, dtype: int64

In [32]:
filter3['attacktype1_txt'] == 'Assassination'

170847    True
170857    True
170888    True
170933    True
170957    True
          ... 
176377    True
176379    True
176404    True
176418    True
176481    True
Name: attacktype1_txt, Length: 358, dtype: bool

In [33]:
filter4 = filter3[filter3['attacktype1_txt'] == 'Assassination']

In [35]:
filter4['city'].value_counts()

city
Marawi                  13
Unknown                 10
Manila                   7
Lamitan                  6
Hinimbangan              4
                        ..
Saloy                    1
La Libertad district     1
Makilala district        1
Nalapaan                 1
Nalus                    1
Name: count, Length: 276, dtype: int64

In [36]:
filter5 = filter4.groupby(['latitude','longitude']).size().reset_index(name='incident_count')

In [37]:
filter5

,latitude,longitude,incident_count
0,5.822439,125.203366,1
1,5.904939,125.567551,1
2,5.933405,121.033258,1
3,5.965086,121.070701,1
4,5.974901,121.033510,1
...,...,...,...
319,17.801519,121.664848,1
320,17.856944,121.492500,1
321,17.889940,121.870264,1
322,17.892728,121.691519,1


In [38]:
from folium.plugins import HeatMap

In [39]:
filter5['normalize_data'] = filter5['incident_count'] - filter5['incident_count'].min() / filter5['incident_count'].max() - filter5['incident_count'].min()
heat_data = filter5[['latitude','longitude','normalize_data']].values.tolist()

In [40]:
my_map = folium.Map(location =[30,40],zoom_start =4)

In [41]:
HeatMap(heat_data).add_to(my_map)


In [42]:
my_map.save('terrorism_bombing.html')
print("Data Successfully Saved to terrorism_bombing.html")

Data Successfully Saved to terrorism_bombing.html


In [43]:
my_map

<h1><strong>Insights</h1>
<h2>Assassination in the Philippines on the first 6 months of 2017</h2>

<h2><strong>1. Urban & Semi-Urban Areas Are Prime Targets</strong> <br><br>

Most visible heat spots align with urban or semi-urban areas with denser populations and political/economic activity—suggesting these zones are strategic targets for high-impact assassinations.</h2>

<h2><strong>2. Metro Manila & CALABARZON Are Assassination Hotspots</strong><br><br>
There is a concentrated red-hot zone in the Metro Manila-Cavite-Laguna corridor, suggesting high-frequency incidents in this densely populated and politically vital area. This could point to an attempt to spread fear in major urban centers.</h2>

<h2><strong>3.  Assassination Activity Spreads Across Central and Eastern Visayas</strong><br><br>
Cities like Cebu, Tacloban, and Ormoc show moderate heat signatures. This reflects targeted killings possibly tied to criminal syndicates or political conflicts, not necessarily full-blown terrorism.</h2>

<h2><strong>4. Assassinations Extend Beyond Conflict Zones</strong><br><br>
    Unlike terrorism bombing hotspots that were mostly concentrated in Mindanao, assassination incidents are more dispersed across the country, suggesting that targeted killings occur even in less traditionally volatile regions.
</h2>

<h2><strong>5.  Basilan and Sulu Archipelago Show Assassination Hotspots</strong><br><br>

Despite being island provinces, Sulu, Basilan, and Tawi-Tawi also show visible activity. This aligns with ongoing threats in those regions from armed groups and extremist factions, where assassinations may serve as control tactics.</h2>